# 2023_06_01

##  Sparse 데이터 문제 해결 방안
현재 학습 데이터셋이 생성되었지만, 데이터의 희소성(sparsity)로 인해 학습이 어렵게 되어있습니다.   
이 문제를 해결하기 위해 두 가지 관점에서 접근하고자 합니다.  

### Temporal Sparsity
시간적인 희소성을 해결하기 위한 다양한 기법을 사용할 계획입니다.

* 이동 평균(Moving Average): 일정 기간 동안의 평균을 계산하여 데이터의 잡음을 줄이고 장기적인 트렌드를 파악하는 기법입니다.
* 지수 이동 평균(Exponential Moving Average, EMA)
* 가중 이동 평균(Weighted Moving Average, WMA)
* ewma

이런 방식들을 PyTorch를 사용하여 학습 데이터의 group_encoder의 transform에 적용하고, 필요한 경우 역함수를 적용하여 원본 시그널을 재구성할 수 있도록 구현하였습니다.  
각 target값이 어떻게 바뀌고 다시 복원되었는지 확인하는 코드를 추가  

다음과 같이 코드를 구성하였습니다.
``` python
# moving average 함수 및 moving average 역함수  
def moving_average_torch(x):
    window_size = 5
    x_torch = torch.tensor(x).unsqueeze(0).unsqueeze(0) # Add two new axes
    weights = torch.ones(window_size, dtype=x_torch.dtype) / window_size
    ma = torch.conv1d(x_torch, weights.unsqueeze(0).unsqueeze(1), padding=window_size // 2)
    return ma.squeeze(1).squeeze(0)
    '''
    """
    Computes the moving average using PyTorch
    """
    x_torch = torch.tensor(x).unsqueeze(0)
    weights = torch.ones(window_size, dtype=x_torch.dtype) / window_size
    ma = torch.conv1d(x_torch, weights.unsqueeze(0).unsqueeze(1), padding=window_size // 2)
    return ma.squeeze(1).squeeze(0)
    '''
def reconstruct_orig(ma_values):
    win_size = 5
    """
    Reconstruct the original signal from moving average values
    """
    re_values = 2 * ma_values - torch.roll(ma_values, win_size - 1)
    return re_values

transformation_dict = {
    'forward': moving_average_torch,
    'reverse': reconstruct_orig
}


def get_training(data , max_prediction_length = 100 , max_encoder_length = 500):
    training_cutoff = data["time_idx"].max() - max_prediction_length
    training = TimeSeriesDataSet(
        ...
        target_normalizer=TorchNormalizer(
            groups = ['group'],
            method='identity',
            transformation=transformation_dict,   # mv 적용 부분  
            center = False,
        ),
        ...
    )
    return training
```


### Spatial Sparsity
공간적인 희소성을 해결하기 위한 접근 방법으로는, geohash 기반의 공간적 가중치 부여 방식을 사용합니다. 사건이 발생한 위치를 중심으로 주변 지역에 가중치를 부여하여 새로운 데이터 칼럼을 생성합니다. 이를 통해 공간적인 정보를 고려하여 학습합니다.

```
0.5 | 0.7 | 0.5
----------------
0.7 | X   | 0.7
----------------
0.5 | 0.7 | 0.5
```

이런 방식을 사용하여 공간적 희소성 문제를 해결하고자 합니다. 이 방법을 적용하기 어려운 경우에는, 타겟 값에 직접 적용 후 학습을 진행합니다.

## Temporal Sparsity 

> TestNotebook : http://202.31.200.194:8888/notebooks/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/TFT/BaseModel_Long%20Beach_5_Temporal%20Sparsity_Test.ipynb

* MA 및 EMWA 적용 plot

![](../Meeting_Img/2023_06_01_1.png)
![](../Meeting_Img/2023_06_01_2.png)


###  plot -> plolty
tft의 plot_prediction을 plolty 버전으로 바꿈   

> http://202.31.200.194:8888/notebooks/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/TFT/BaseModel_Long%20Beach_5_Temporal%20Sparsity_visualization.ipynb

![](../Meeting_Img/2023_05_25_1.png)
![](../Meeting_Img/2023_05_25_2.png)


# transfomation & inv transformation 

* MA
* EMWA
* Passion pdf
* Noraml Dist pdf

pdf 방식은 연속된 방식으로 데이티를 늘리는 방식으로 학습에 더 용이할 것으로 생각되어서 만들어봄  

In [16]:
from IPython.display import display, IFrame
import os

# Get the list of all files in the directory
all_files = os.listdir('../Traffic_Accient/TFT/Transformation_Html')

# Filter out all non-html files
html_files = [file for file in all_files if file.endswith('.html')]

# Display each HTML file
for file in html_files:
    display(IFrame(src=f'../Traffic_Accient/TFT/Transformation_Html/{file}', width=1000, height=500))


# 학습용 코드 

pytorch-forecasting을 업데이트를 해서 기존의 코드를 사용하면 에러남    
코드 수정후 window_size와 lr별로 turning 진행  

> MA : http://202.31.200.194:8888/notebooks/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/TFT/BaseModel_Long%20Beach_5_Temporal%20Sparsity_MA.ipynb

> EWMA : http://202.31.200.194:8888/notebooks/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/TFT/BaseModel_Long%20Beach_5_Temporal%20Sparsity_EMWA.ipynb

> Possion : http://202.31.200.12:10614/notebooks/thkim/SEO/Emergency_Demand/BaseModel_Long%20Beach_5_Temporal%20Sparsity_Possion.ipynb

> Normal Dist : http://202.31.200.12:10614/notebooks/thkim/SEO/Emergency_Demand/BaseModel_Long%20Beach_5_Temporal%20Sparsity_Normal_Dist.ipynb

# 학습 결과 코드
early_stopping이 끝난 시간 , 실행시간등을 기록

> http://202.31.200.194:8888/notebooks/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/TFT/Transform_Result.ipynb#

> http://202.31.200.12:10614/notebooks/thkim/SEO/Emergency_Demand/Transform_Result.ipynb